In [1]:
import pickle
import time
import pandas as pd
import numpy as np
from itertools import combinations
from collections import defaultdict
import csv

In [2]:
clst_file = "../ppi_ml/results/ppi_clustering/final_cmplx_annots_011823.csv"
apms_pkl = "../ppi_ml/data/apms/apms_scores_dict.pkl"
apms_fmat_file = "../ppi_ml/data/apms/featmat_humap"
outdir = "../ppi_ml/data/apms/og_scores/"

In [3]:
with open(apms_pkl, 'rb') as handle:
    apms_dict = pickle.load(handle)

In [4]:
df = pd.read_csv(clst_file)
ogs = df['ID'].tolist()
len(ogs)

2778

In [5]:
cmplx_groups = df.groupby('granulated_cmplx_name')
cmplx_groups.groups.keys()

dict_keys(['(ER)-localized multiprotein complex 1', '(ER)-localized multiprotein complex 2', '(ER)-localized multiprotein complex 3', '19S Proteasome', '2-oxoglutarate dehydrogenase complex', '20S Proteasome', '26S Proteasome', '3-phosphoinositide-dependent protein kinase', '37S mitochondrial small ribosomal subunit', '3M complex', '43S preinitiation complex', "5'-deoxynucleotidase", '7SK RNP complex', '90S preribosome', 'ABC transporter complex', 'ABCG sterol transporter complex', 'ABITRAM-PUS10 cell death complex', 'ADP-ribosylation factor GTPase-activating protein', 'AGO/TrxG complex', 'AMP deaminase', 'AMPA glutamate receptor complex', 'AMPA glutamate receptor complex 1', 'AMPA glutamate receptor complex 2', 'AMPK complex', 'ASAP complex', 'ATAC complex', 'ATR-ATRIP DNA damage-sensing kinase complex', 'Acetolactate synthase complex 1', 'Acetolactate synthase complex 2', 'Adaptor complex AP-1', 'Adaptor complex AP-2', 'Adaptor complex AP-3', 'Afadin-beta-catenin complex', 'Aminoacyl

In [6]:
#for c, p in cmplx_groups:
    #print(c, '\n', p['ID'].tolist())

In [8]:
pair_dict = dict()
issues = []
for cmplx, df in cmplx_groups:
    ogs = df['ID'].tolist()
    if len(ogs) > 1:
        for i in ogs:
            self_interaction = frozenset({i, i})
        pairs = [frozenset({i, j}) for i,j in list(combinations(ogs, 2))]
        pairs.append(self_interaction)
        if cmplx not in pair_dict.keys():
            pair_dict.update({cmplx: pairs})
        else:
            issues.append(cmplx)
            pair_dict.pop(cmplx)

In [9]:
all_pairs = []
for c, p in pair_dict.items():
    for pair in p:
        if pair not in all_pairs:
            all_pairs.append(pair)
len(all_pairs)

19367

In [42]:
out_dict = dict()
for k, v in pair_dict.items():
    vals = []
    for pair in v:
        scores = apms_dict.get(pair)
        # oh man, this is ugly
        # ... but it works ¯\_(ツ)_/¯
        if scores:
            for score_list in scores:
                for i, s in enumerate(score_list):
                    if s == '':
                        score_list[i] = np.nan
        out_dict[pair] = scores
    if not vals:
        continue

In [43]:
out_dict

{frozenset({'KOG0020',
            'KOG0730'}): [['106.1875',
   '0.0',
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   '2.3881698017',
   '1.0',
   '6.5',
   '0.035',
   '-0.865',
   '-0.6358835',
   '0.957759',
   '8.0',
   None,
   None,
   '6.177897327198999',
   '6.5',
   '6.05435897166',
   '36.0',
   '5.77111341332',
   '24.0',
   '3.25156888802',
   '10.0',
   '1.79611',
   '0.0',
   '0.0',
   '1.79611',
   '7.45725773959',
   '10.0',
   '4.81421032956',
   '7.0',
   '321.362170611',
   '185.0',
   '1.7807923174319198',
   '21.0',
   '4.2086402670345',
   '8.5',
   '0.838603306625',
   '2.0',
   '237.95390510428',
   '141.0']],
 frozenset({'KOG0020',
            'KOG4563'}): [['0.0',
   '0.0',
   nan,
   nan,
   nan,
   nan,
   nan,
   nan,
   nan,
   nan,
   nan,
   nan,
   nan,
   '15.0',
   '0.05',
   '-0.09',
   '-0.157702',
   '0.960362',
   '18.0',
   nan,
   nan,
   '0.891453940996',
   '2.0',
   '0.688591057219',
   '35.0',
   '0.9494

In [44]:
with open('/stor/work/Marcotte/project/rmcox/leca/ppi_ml/data/apms/clustered_og_scores_dict.pkl', 'wb') as handle:
    pickle.dump(out_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [45]:
with open(apms_fmat_file, 'r') as f:
    cols = f.readline().strip('\n').split(',')
    score_cols = cols[1:]

In [46]:
out_dict

{frozenset({'KOG0020',
            'KOG0730'}): [['106.1875',
   '0.0',
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   '2.3881698017',
   '1.0',
   '6.5',
   '0.035',
   '-0.865',
   '-0.6358835',
   '0.957759',
   '8.0',
   None,
   None,
   '6.177897327198999',
   '6.5',
   '6.05435897166',
   '36.0',
   '5.77111341332',
   '24.0',
   '3.25156888802',
   '10.0',
   '1.79611',
   '0.0',
   '0.0',
   '1.79611',
   '7.45725773959',
   '10.0',
   '4.81421032956',
   '7.0',
   '321.362170611',
   '185.0',
   '1.7807923174319198',
   '21.0',
   '4.2086402670345',
   '8.5',
   '0.838603306625',
   '2.0',
   '237.95390510428',
   '141.0']],
 frozenset({'KOG0020',
            'KOG4563'}): [['0.0',
   '0.0',
   nan,
   nan,
   nan,
   nan,
   nan,
   nan,
   nan,
   nan,
   nan,
   nan,
   nan,
   '15.0',
   '0.05',
   '-0.09',
   '-0.157702',
   '0.960362',
   '18.0',
   nan,
   nan,
   '0.891453940996',
   '2.0',
   '0.688591057219',
   '35.0',
   '0.9494

In [47]:
df = pd.DataFrame([out_dict.keys(), out_dict.values()]).T
df[['ID1', 'ID2']] = pd.DataFrame(df[0].tolist(), index=df.index)

In [48]:
df

,0,1,ID1,ID2
0,"(KOG0020, KOG0730)","[[106.1875, 0.0, None, None, None, None, None,...",KOG0020,KOG0730
1,"(KOG0020, KOG4563)","[[0.0, 0.0, nan, nan, nan, nan, nan, nan, nan,...",KOG0020,KOG4563
2,"(KOG0730, KOG4563)","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...",KOG0730,KOG4563
3,(KOG4563),"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...",KOG4563,None
4,"(KOG3544, KOG0190)","[[nan, nan, nan, nan, nan, nan, nan, 5.0267844...",KOG3544,KOG0190
...,...,...,...,...
19362,"(KOG4380, KOG0115)","[[0.0, 0.0, nan, nan, nan, nan, nan, nan, nan,...",KOG4380,KOG0115
19363,"(KOG1914, KOG3833)","[[0.0, 0.0, nan, nan, nan, nan, nan, nan, nan,...",KOG1914,KOG3833
19364,"(KOG4380, KOG3833)","[[0.0, 0.0, nan, nan, nan, nan, nan, nan, nan,...",KOG4380,KOG3833
19365,"(KOG1914, KOG4380)","[[0.0, 0.0, nan, nan, nan, nan, nan, nan, nan,...",KOG1914,KOG4380


In [49]:
df2 = df[1].apply(pd.Series, dtype=float)

In [50]:
df_scores = pd.DataFrame(df2[0].to_list(), columns=score_cols)

In [51]:
df_final = df[['ID1', 'ID2']].join(df_scores)

In [52]:
df_final

,ID1,ID2,ext_Dm_guru,ext_Hs_malo,entropy_orig9k,zscore_orig9k,nwdscore_orig9k,plate_zscore_orig9k,uPeps_orig9k,neg_ln_pval,...,neg_ln_pval_treiber_hygeo_gt4,pair_count_treiber_hygeo_gt4,neg_ln_pval_youn_hygeo,pair_count_youn_hygeo,neg_ln_pval_youn_hygeo_gt2,pair_count_youn_hygeo_gt2,neg_ln_pval_youn_hygeo_gt4,pair_count_youn_hygeo_gt4,neg_ln_pval_treiber_hygeo_gt2,pair_count_treiber_hygeo_gt2
0,KOG0020,KOG0730,106.1875,0.0,None,None,None,None,None,None,...,321.362170611,185.0,1.7807923174319198,21.0,4.2086402670345,8.5,0.838603306625,2.0,237.95390510428,141.0
1,KOG0020,KOG4563,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.79477044991,82.0,0.574994780399,45.0,1.43854062237,12.0,0.858966105654,3.0
2,KOG0730,KOG4563,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.415840928281,2.0,NaN,NaN,NaN,NaN,NaN,NaN
3,KOG4563,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,KOG3544,KOG0190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.026784491335,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19362,KOG4380,KOG0115,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.4487079031100004e-12,2.0,NaN,NaN,NaN,NaN,NaN,NaN,-2.22044604925e-16,4.0
19363,KOG1914,KOG3833,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.09752412691,1.0,0.229261697311,15.0,0.300876679894,8.0,0.0194486627932,1.0,0.865448414471,3.0
19364,KOG4380,KOG3833,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,-5.39124300758e-13,1.0,NaN,NaN,NaN,NaN,NaN,NaN,-2.22044604925e-16,4.0
19365,KOG1914,KOG4380,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
def match_cmplx(x, my_dict):
    for cmplx, pair_list in my_dict.items():
        if any(pair == x for pair in pair_list):
            return(cmplx)

In [54]:
df_final['cmplx'] = [match_cmplx(frozenset({i, j}), pair_dict) for i, j in zip(df['ID1'], df['ID2'])]
df_final

,ID1,ID2,ext_Dm_guru,ext_Hs_malo,entropy_orig9k,zscore_orig9k,nwdscore_orig9k,plate_zscore_orig9k,uPeps_orig9k,neg_ln_pval,...,pair_count_treiber_hygeo_gt4,neg_ln_pval_youn_hygeo,pair_count_youn_hygeo,neg_ln_pval_youn_hygeo_gt2,pair_count_youn_hygeo_gt2,neg_ln_pval_youn_hygeo_gt4,pair_count_youn_hygeo_gt4,neg_ln_pval_treiber_hygeo_gt2,pair_count_treiber_hygeo_gt2,cmplx
0,KOG0020,KOG0730,106.1875,0.0,None,None,None,None,None,None,...,185.0,1.7807923174319198,21.0,4.2086402670345,8.5,0.838603306625,2.0,237.95390510428,141.0,(ER)-localized multiprotein complex 1
1,KOG0020,KOG4563,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.79477044991,82.0,0.574994780399,45.0,1.43854062237,12.0,0.858966105654,3.0,(ER)-localized multiprotein complex 1
2,KOG0730,KOG4563,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.415840928281,2.0,NaN,NaN,NaN,NaN,NaN,NaN,(ER)-localized multiprotein complex 1
3,KOG4563,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,KOG3544,KOG0190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.026784491335,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(ER)-localized multiprotein complex 2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19362,KOG4380,KOG0115,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,-2.22044604925e-16,4.0,tRNA-splicing ligase complex
19363,KOG1914,KOG3833,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.229261697311,15.0,0.300876679894,8.0,0.0194486627932,1.0,0.865448414471,3.0,tRNA-splicing ligase complex
19364,KOG4380,KOG3833,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,-2.22044604925e-16,4.0,tRNA-splicing ligase complex
19365,KOG1914,KOG4380,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tRNA-splicing ligase complex


In [55]:
df_final.to_csv('../ppi_ml/data/apms/clustered_og_scores.csv')

In [56]:
fmat_score_cols = [25, 42, 40, 14, 46]
scols = [x+2 for x in fmat_score_cols]
scols

[27, 44, 42, 16, 48]

In [57]:
scores_subset = df_final.iloc[:, scols]
df_subset = df_final[['ID1', 'ID2', 'cmplx']].join(scores_subset)
df_subset

,ID1,ID2,cmplx,neg_ln_pval_cilium_hygeo_avgspec2,pair_count_youn_hygeo_gt2,pair_count_youn_hygeo,nwdscore_bioplex2,pair_count_treiber_hygeo_gt2
0,KOG0020,KOG0730,(ER)-localized multiprotein complex 1,5.77111341332,8.5,21.0,0.035,141.0
1,KOG0020,KOG4563,(ER)-localized multiprotein complex 1,0.949493075471,45.0,82.0,0.05,3.0
2,KOG0730,KOG4563,(ER)-localized multiprotein complex 1,NaN,NaN,2.0,NaN,NaN
3,KOG4563,None,None,NaN,NaN,NaN,0.57,NaN
4,KOG3544,KOG0190,(ER)-localized multiprotein complex 2,1.8892968242050001,NaN,NaN,0.0642857142857143,NaN
...,...,...,...,...,...,...,...,...
19362,KOG4380,KOG0115,tRNA-splicing ligase complex,0.0,NaN,NaN,0.06,4.0
19363,KOG1914,KOG3833,tRNA-splicing ligase complex,NaN,8.0,15.0,NaN,3.0
19364,KOG4380,KOG3833,tRNA-splicing ligase complex,9.71994747015,NaN,NaN,NaN,4.0
19365,KOG1914,KOG4380,tRNA-splicing ligase complex,NaN,NaN,NaN,NaN,NaN


In [58]:
df_subset.to_csv('../ppi_ml/data/apms/clustered_og_scores_subset.csv', index=False)

These scores are shockingly low & sparse among pairs in the final clustered data set. Trying some things to make sure nothing went wrong during the mapping script:

In [ ]:
test_pair = set(['KOG2604', 'KOG4673'])
print(f'Evaluating out_dict for {test_pair}...')
for pair, score_list in out_dict.items():
    if test_pair == pair:
        print(pair, ':')
        print(score_list)

print()
print(f'Evaluating apms_dict for {test_pair} ...')
for pair, score_list in apms_dict.items():
    if test_pair == pair:
        print(pair, ':')
        print(score_list)

Evaluating out_dict for {'KOG2604', 'KOG4673'}...
frozenset({'KOG2604', 'KOG4673'}) :
[['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '5.10657685349', '2', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]

Evaluating apms_dict for {'KOG2604', 'KOG4673'} ...
frozenset({'KOG2604', 'KOG4673'}) :
[['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '5.10657685349', '2', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]

Featmap vals for KOG2604 + KOG4673:
KOG2604 KOG4673,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.10657685349,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

Hmmmmm this looks good... let's try one more: KOG0178 + KOG0176 (PSMA4 + PSMA5)

In [ ]:
test_pair = set(['KOG0178', 'KOG0176'])
print(f'Evaluating out_dict for {test_pair}...')
for pair, score_list in out_dict.items():
    if test_pair == pair:
        print(pair, ':')
        print(score_list)

print()
print(f'Evaluating apms_dict for {test_pair} ...')
for pair, score_list in apms_dict.items():
    if test_pair == pair:
        print(pair, ':')
        print(score_list)

In [ ]:
x = 'KOG2604'
print()
print(f'Evaluating apms_dict for {x} ...')
for pair, score_list in apms_dict.items():
    if any(x in p for p in pair):
        print(pair, ':')
        print(score_list)

In [ ]:
x = 'KOG0176'
print()
print(f'Evaluating apms_dict for {x} ...')
for pair, score_list in apms_dict.items():
    if any(x in p for p in pair):
        print(pair, ':')
        print(score_list)